In [1]:
import sys
import json
params = {}
default = {'training_folder': '../../scenarios/houston/', 'params': params} # Default execution parameters
manual, argv = (True, default) if 'ipykernel' in sys.argv[0] else (False, dict(default, **json.loads(sys.argv[1])))
print(argv)


{'training_folder': '../../scenarios/houston/', 'params': {}}


In [2]:
import os
import time
import geopandas as gpd
import pandas as pd
sys.path.insert(0, r'../../../quetzal') # Add path to quetzal
from sklearn.neighbors import NearestNeighbors
from numba import jit, njit
import numba as nb
from quetzal.model import stepmodel
from shapely.geometry import LineString
from quetzal.io.gtfs_reader.importer import get_epsg
from quetzal.io import excel
on_lambda = bool(os.environ.get('AWS_EXECUTION_ENV'))
num_cores = nb.config.NUMBA_NUM_THREADS
print('num cores:',num_cores)

io_engine= 'pyogrio' if on_lambda else 'pyogrio' #or fiona

PyTables is not installed. No support for HDF output.
num cores: 8


In [3]:
base_folder = argv['training_folder']
input_folder = os.path.join(base_folder,'inputs/')
pt_folder  = os.path.join(input_folder,'pt/')
road_folder = os.path.join(input_folder,'road/')
od_folder =  os.path.join(input_folder,'od/')
params_folder = input_folder if on_lambda else '../../inputs/'

output_folder = os.path.join(base_folder,'outputs/')
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
    
model_folder = os.path.join(input_folder, 'model/')


In [4]:
sm = stepmodel.read_zippedpickles(model_folder +'aon_los')


zones: 100%|██████████| 3/3 [00:00<00:00, 39.48it/s]


In [5]:
var = excel.read_var(file=os.path.join(params_folder,'parameters.xlsx'), scenario='base', return_ancestry=False) 
motor_rate = var['generation']['motor_rate'] 
pt_rate = (1-motor_rate)


In [6]:

sm.zones['population'].sum()

7149642

In [7]:
trip_per_person = 0.5 ## Param

sm.zones['emission'] =  sm.zones['population'] * trip_per_person##  
sm.zones['attraction'] =  sm.zones['jobs'] ##

car = sm.copy()
car.zones['attraction']= car.zones['attraction'] * trip_per_person
car.zones['emission'] *= motor_rate  ###

pt_captive = sm.copy()
pt_captive.zones['emission']= pt_captive.zones['emission']
pt_captive.zones['emission'] *= pt_rate

In [8]:
imp_matrix = car.car_los[ 
    ['origin', 'destination', 'time']
].set_index(['origin','destination']).unstack()
imp_matrix = imp_matrix.replace(0, 120) ### Param

car.step_distribution(deterrence_matrix=imp_matrix) 


## Param

In [9]:
imp_matrix

time                                            \
destination        zone_0        zone_1       zone_10      zone_100   
origin                                                                
zone_0         120.000000   3445.684922   2020.607366   3099.640056   
zone_1        3410.705945    120.000000   3392.890681   2679.658128   
zone_10       2025.797768   3442.219786    120.000000   2669.899368   
zone_100      3072.918457   2700.549512   2650.722582    120.000000   
zone_101     12558.721104  11220.215261  12540.905839  11268.545963   
...                   ...           ...           ...           ...   
zone_95      15596.676345  16786.916376  16672.598891  16830.803679   
zone_96       2191.367492   4831.428431   2739.417140   4355.785650   
zone_97       3520.508644   4710.748676   4596.431191   4754.635978   
zone_98      37267.057205  39907.118144  38047.920654  39664.289164   
zone_99       4029.550821   5219.790852   5105.473367   5263.678155   

                                                                     \
destination      zone_101      zone_102      zone_103      zone_104   
origin                                                                
zone_0       12542.204179   2708.945190   3356.558840   1432.576731   
zone_1       11200.796616   3871.758320   2877.097996   2496.183887   
zone_10      12526.188286   3802.394934   2926.818152   1597.392924   
zone_100     11177.427305   3954.058145   1522.076913   2316.680118   
zone_101       120.000000  12620.459577  11353.854191  11644.199045   
...                   ...           ...           ...           ...   
zone_95      25159.554265  13703.041966  16981.574780  15399.321111   
zone_96      13927.947688   4037.439475   4612.704434   2846.580647   
zone_97      13263.286701   1626.874266   4905.407080   3323.153411   
zone_98      49003.637401  38769.345933  39921.207948  37922.270360   
zone_99      13446.794771   2055.782075   5414.449256   3832.195587   

                                         ...                              \
destination      zone_105      zone_106  ...       zone_90       zone_91   
origin                                   ...                               
zone_0        4323.567656  10071.853923  ...   3504.714896  20722.337132   
zone_1        4711.210486   9484.976564  ...   4196.883177  20893.545408   
zone_10       5133.452501  10699.970854  ...   4557.742940  21348.966929   
zone_100      4946.884284   9635.265289  ...   4650.719800  21166.052352   
zone_101     11748.340706  15681.287693  ...  12482.387074  27967.508774   
...                   ...           ...  ...           ...           ...   
zone_95      15796.583320  22498.335319  ...  14981.308689  32560.367895   
zone_96       5692.883597  11457.597431  ...   4833.209181  22108.080641   
zone_97       3900.315755  10602.067754  ...   3085.041124  20664.100330   
zone_98      40566.018915  46533.287144  ...  39565.115640  57183.770354   
zone_99       4083.823826  10785.575825  ...   3268.549194  20847.608401   

                                                                     \
destination       zone_92       zone_93       zone_94       zone_95   
origin                                                                
zone_0        6820.146782   4907.599345   4770.051933  15602.348637   
zone_1        9435.154496   5120.668471   6665.632525  16765.161767   
zone_10       7611.647868   4482.234308   4077.141737  16695.798380   
zone_100      9200.198890   2711.628751   5735.111679  16847.461591   
zone_101     18583.169654  13653.647013  15779.046289  25181.502884   
...                   ...           ...           ...           ...   
zone_95      21052.058615  19236.234591  19422.043458    120.000000   
zone_96       6496.290703   6194.683465   5094.175278  16930.842922   
zone_97       8975.890915   7160.066891   7345.875758  13130.937629   
zone_98      37081.574249  41474.027485  40269.315893  51427.953240   
zone_99       9377.327466   7669.109067   7

In [10]:
imp_matrix = pt_captive.pt_los[ 
    ['origin', 'destination', 'gtime']
].set_index(['origin','destination']).unstack()
imp_matrix = imp_matrix.replace(0, 120) #####

In [12]:
imp_matrix

gtime                                             \
destination        zone_0        zone_1       zone_10       zone_100   
origin                                                                 
zone_0         120.000000  12078.042640  18365.817067   26982.258048   
zone_1       11046.909407    120.000000  21563.728921   29683.175354   
zone_10      18247.493650  23520.327198    120.000000   38424.542606   
zone_100     27614.124068  29165.835531  38118.083460     120.000000   
zone_101     63299.454028  59956.181679  73816.273542   69889.083005   
...                   ...           ...           ...            ...   
zone_95      48695.356670  52850.635588  59138.410014   67754.850996   
zone_96      23953.924567  33824.546087  35384.651598   48728.761495   
zone_97      32217.627072  36372.905990  42660.680416   51277.121398   
zone_98      78267.588156  86113.005664  91977.298210  101017.221072   
zone_99      23921.704500  28076.983418  34364.757845   42981.198826   

                                                                      \
destination       zone_101      zone_102      zone_103      zone_104   
origin                                                                 
zone_0        62526.973599  15779.455549  16446.981531   6814.105732   
zone_1        59848.675044  17828.801035  19549.429587   9940.118867   
zone_10       73969.258158  26399.796956  27889.266090  16775.495326   
zone_100      69845.061319  34630.572658  14197.630848  25439.434215   
zone_101        120.000000  70081.345656  68397.234455  62192.663488   
...                    ...           ...           ...           ...   
zone_95      103299.566547  35803.263792  57219.574479  47795.228122   
zone_96       84273.477046  37525.958996  38193.484979  28042.851611   
zone_97       86821.836949  18301.113942  40741.844881  31317.498524   
zone_98      136561.936623  86440.514145  90481.944555  79768.264652   
zone_99       78525.914377  11920.179028  32445.922310  23021.575952   

                                           ...                               \
destination       zone_105       zone_106  ...       zone_90        zone_91   
origin                                     ...                                
zone_0        44292.813849   68474.600889  ...  22946.025979   82684.019219   
zone_1        46118.689866   66298.029959  ...  24995.371464   84509.895236   
zone_10       55290.881047   79916.885447  ...  33566.367386   93682.086417   
zone_100      63100.069590   88226.867283  ...  41797.143087  101491.274960   
zone_101      76196.209994   55529.024550  ...  77247.916085  107062.033923   
...                    ...            ...  ...           ...            ...   
zone_95       72056.019269  109247.193837  ...  49993.530272  117656.981751   
zone_96       66039.317296   90221.104336  ...  44692.529426  104430.522666   
zone_97       57692.462745   92769.464239  ...  35629.973748  103293.425227   
zone_98      117856.215059  142509.563913  ...  96009.337535  156247.420429   
zone_99       46383.716055   84473.541667  ...  24321.227058   91984.678537   

                                                                         \
destination        zone_92        zone_93        zone_94        zone_95   
origin                                                                    
zone_0        58366.258442   28594.819678   43604.152476   48970.357188   
zone_1        63673.073575   31360.721194   46802.064331   51019.702674   
zone_10       71458.397165   39337.086930   45309.468130   59590.698595   
zone_100      80453.056733   10704.737259   63356.418870   67821.474296   
zone_101     115925.618196   77958.575298   99054.608951  103272.247295   
...                    ...            ...            ...            ...   
zone_95       97214.311351   69215.830449   84376.745424     120.000000   
zone_96       47368.625228   50341.323125   56781.221007   70716.860635   
zone_97       81769.707709   52738.100851   67899.015826   18862.52377

In [13]:
pt_captive.step_distribution(deterrence_matrix=imp_matrix)

In [14]:
sm.volumes = pd.merge(
    car.volumes,
    pt_captive.volumes,
    on=['origin', 'destination'],
    suffixes=[ '_car_owner', '_pt_captive']
).rename(columns={'volume_car_owner': 'car_owner', 'volume_pt_captive': 'pt_captive'})
sm.volumes

,origin,destination,car_owner,pt_captive
0,zone_0,zone_0,0.297001,0.015988
1,zone_0,zone_1,41.714743,7.895056
2,zone_0,zone_2,8.048780,2.684712
3,zone_0,zone_3,22.152330,5.966792
4,zone_0,zone_4,25.290300,6.470099
...,...,...,...,...
249995,zone_499,zone_495,0.899564,0.136971
249996,zone_499,zone_496,0.577349,0.138062
249997,zone_499,zone_497,10.194185,1.955932
249998,zone_499,zone_498,0.297271,0.070590


In [15]:
sm.sample_volumes(bidimentional_sampling=True, fit_sums=True, sample_weight={'car_owner':50, 'pt_captive':50}, sample_size=None, inplace=True)

In [16]:
len(sm.volumes) - sm.volumes.isnull().sum()

origin         67776
destination    67776
car_owner      60503
pt_captive     22042
dtype: int64

In [17]:
sm.volumes


,origin,destination,car_owner,pt_captive
0,zone_0,zone_1,25.171631,NaN
1,zone_0,zone_104,100.686525,50.702857
2,zone_0,zone_109,50.193486,NaN
3,zone_0,zone_120,25.171631,NaN
4,zone_0,zone_121,25.171631,NaN
...,...,...,...,...
67771,zone_99,zone_81,25.028070,NaN
67772,zone_99,zone_82,25.028070,NaN
67773,zone_99,zone_85,25.028070,NaN
67774,zone_99,zone_87,25.062561,25.475000


In [18]:
sm.to_zippedpickles(model_folder +'distribution', only_attributes= ['volumes']) #


volumes: 100%|██████████| 10/10 [00:00<00:00, 303.61it/s]
